In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from load_data import load_data, load_SIAM_NRG
from plotting_routines import generate_filename_fRG, generate_label
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import MaxNLocator
from itertools import product
import heapq

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

path = "C:/Users/aguir/Documents/Data_Thesis_Local/"

In [ ]:
with h5py.File(path + 'SIAM_NRG.h5', 'r') as f:
    print(f.keys())

In [ ]:
def log_subst(x):
    return np.log10(1 + x*200.);

In [ ]:
def log_resubst(X):
    return 1./200.*(pow(10, X) - 1);

In [ ]:
def reconstruct_flow_grid(Lambda_ini, Lambda_fin, nODE):
    X_ini = log_subst(Lambda_ini)
    X_fin = log_subst(Lambda_fin)
    X = np.linspace(X_ini, X_fin, nODE, endpoint=True)
    return log_resubst(X)

In [ ]:
def log_derivative_of_log_f(f, x):
    logx = np.log10(x)
    logf = np.log10(f)
    
    dlogx = np.zeros(len(logx)-1)
    dlogf = np.zeros(len(logf)-1)
    for i in range(len(dlogx)):
        dlogx[i] = logx[i+1] - logx[i]
        dlogf[i] = logf[i+1] - logf[i]
    return dlogf/dlogx

In [ ]:
def interpolate(f, grid, x):
    diff = abs(grid-x)
    smallest_diff = min(diff)
    second_smallest_diff = heapq.nsmallest(2, diff)[-1]
    
    i1 = np.where(diff==smallest_diff)[0][0]
    i2 = np.where(diff==second_smallest_diff)[0][0]
    
    f1 = f[i1]
    f2 = f[i2]
    
    return f1 + (f2-f1)/(grid[i2]-grid[i1]) * (x-grid[i1])

In [ ]:
U_NRG_list = [0,0.05, 0.1, 0.2, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 3]
U_NRG_vector = np.array(U_NRG_list)

Deltas_NRG = 1/(2*U_NRG_vector)

In [ ]:
Lambdas = reconstruct_flow_grid(1000, 0, 50)
Gamma = 1./3.
Deltas_fRG = (Lambdas + Gamma)/2
U_fRG = 1/(2*Deltas_fRG)
#U_fRG = np.append(U_fRG, U_NRG)
#U_fRG.sort()
#Deltas_fRG = 1/(2*U_fRG)


In [ ]:
K1 = generate_filename_fRG(path, 1, 1)
K1_sf = generate_filename_fRG(path, 1, 1, sf=True)
K2_1L = generate_filename_fRG(path, 2,1)
K2_2L = generate_filename_fRG(path, 2,2)

In [ ]:
filenames = [K1, K1_sf, K2_1L, K2_2L]
labels = generate_label(path, filenames)

In [ ]:
SER_0_K1    = np.zeros(len(U_fRG))
SER_0_K1_sf = np.zeros(len(U_fRG))
SER_0_K2_1L = np.zeros(len(U_fRG))
SER_0_K2_2L = np.zeros(len(U_fRG))

K1aR_0_K1    = np.zeros(len(U_fRG))
K1aR_0_K1_sf = np.zeros(len(U_fRG))
K1aR_0_K2_1L = np.zeros(len(U_fRG))
K1aR_0_K2_2L = np.zeros(len(U_fRG))

K1pR_0_K1    = np.zeros(len(U_fRG))
K1pR_0_K1_sf = np.zeros(len(U_fRG))
K1pR_0_K2_1L = np.zeros(len(U_fRG))
K1pR_0_K2_2L = np.zeros(len(U_fRG))

K1tR_0_K1    = np.zeros(len(U_fRG))
K1tR_0_K1_sf = np.zeros(len(U_fRG))
K1tR_0_K2_1L = np.zeros(len(U_fRG))
K1tR_0_K2_2L = np.zeros(len(U_fRG))

Gamma_NRG = 1

for i in range(len(U_fRG)):
    U_NRG = U_fRG[i]
    Delta_NRG = Gamma_NRG/2
    
    _, _, _, _, _, _, _, Delta_list, _, _, v_list, Sigma_list,\
    _, w_list, K1a_list, K1p_list, K1t_list, _, _, _, _, _ = load_data(U_NRG, Delta_NRG, filenames)
    
    v = np.array(v_list[0])
    w = np.array(v_list[0])

    iv = np.where(v==0)[0][0]
    iw = np.where(w==0)[0][0]
    
    SER_0_K1[i]    = (Sigma_list[0][0][iv+1].real - Sigma_list[0][0][iv-1].real)/((v[iv+1]) - v[iv-1])
    SER_0_K1_sf[i] = (Sigma_list[1][0][iv+1].real - Sigma_list[1][0][iv-1].real)/((v[iv+1]) - v[iv-1])
    SER_0_K2_1L[i] = (Sigma_list[2][0][iv+1].real - Sigma_list[2][0][iv-1].real)/((v[iv+1]) - v[iv-1])
    SER_0_K2_2L[i] = (Sigma_list[3][0][iv+1].real - Sigma_list[3][0][iv-1].real)/((v[iv+1]) - v[iv-1])
    
    K1aR_0_K1[i]    = K1a_list[0][0][iw].real
    K1aR_0_K1_sf[i] = K1a_list[1][0][iw].real
    K1aR_0_K2_1L[i] = K1a_list[2][0][iw].real
    K1aR_0_K2_2L[i] = K1a_list[3][0][iw].real
    
    K1pR_0_K1[i]    = K1p_list[0][0][iw].real
    K1pR_0_K1_sf[i] = K1p_list[1][0][iw].real
    K1pR_0_K2_1L[i] = K1p_list[2][0][iw].real
    K1pR_0_K2_2L[i] = K1p_list[3][0][iw].real
    
    K1tR_0_K1[i]    = K1t_list[0][0][iw].real
    K1tR_0_K1_sf[i] = K1t_list[1][0][iw].real
    K1tR_0_K2_1L[i] = K1t_list[2][0][iw].real
    K1tR_0_K2_2L[i] = K1t_list[3][0][iw].real
    
    

In [ ]:
chi_sp_K1 = -K1aR_0_K1
chi_sp_K1_sf = -K1aR_0_K1_sf
chi_sp_K2_1L = -K1aR_0_K2_1L
chi_sp_K2_2L = -K1aR_0_K2_2L

chi_ch_K1 = 2*K1tR_0_K1 - K1aR_0_K1
chi_ch_K1_sf = 2*K1tR_0_K1_sf - K1aR_0_K1_sf
chi_ch_K2_1L = 2*K1tR_0_K2_1L - K1aR_0_K2_1L
chi_ch_K2_2L = 2*K1tR_0_K2_2L - K1aR_0_K2_2L

In [ ]:
SE_NRG = np.zeros(len(U_NRG_vector))
K1a_NRG = np.zeros(len(U_NRG_vector))
K1p_NRG = np.zeros(len(U_NRG_vector))
K1t_NRG = np.zeros(len(U_NRG_vector))

for i in range(len(U_NRG_list)):
    U_NRG = U_NRG_list[i]
    
    _, w_NRG, _, SE_re, _, K1a_re, _, K1p_re, _, K1t_re, _ = load_SIAM_NRG(U_NRG, path)
    
    iwNRG = int(len(w_NRG)/2)
    
    ivNRG_m = np.where(w_NRG > v[iv-1])[0][0]
    ivNRG_p = np.where(w_NRG > v[iv+1])[0][0]
    
    SE_NRG[i] = (SE_re[ivNRG_p] - SE_re[ivNRG_m])/(w_NRG[ivNRG_p] - w_NRG[ivNRG_m])
    K1a_NRG[i] = (K1a_re[iwNRG] + K1a_re[iwNRG+1])*0.5*U_NRG**2  
    K1p_NRG[i] = (K1p_re[iwNRG] + K1p_re[iwNRG+1])*0.5*U_NRG**2
    K1t_NRG[i] = (K1t_re[iwNRG] + K1t_re[iwNRG+1])*0.5*U_NRG**2

    

In [ ]:
chi_sp_NRG = -K1a_NRG * Deltas_NRG *2
chi_ch_NRG = (2*K1t_NRG - K1a_NRG) * Deltas_NRG *2


In [ ]:
Z_K1    = 1/(1-SER_0_K1)
Z_K1_sf = 1/(1-SER_0_K1_sf)
Z_K2_1L = 1/(1-SER_0_K2_1L)
Z_K2_2L = 1/(1-SER_0_K2_2L)
Z_NRG = 1/(1-SE_NRG)

In [ ]:
fs = 32
lw=4

abss = True
diff = False


if abss:
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))

    ax[0].plot(1/Deltas_fRG, (chi_sp_K1*np.pi*Deltas_fRG), '.-', label=labels[0], color='tab:blue', linewidth=lw)
    ax[0].plot(1/Deltas_fRG, (chi_sp_K2_1L*np.pi*Deltas_fRG), '.-', label=labels[2], color='tab:green', linewidth=lw)
    ax[0].plot(1/Deltas_fRG, (chi_sp_K2_2L*np.pi*Deltas_fRG), '.-', label=labels[3], color='tab:red', linewidth=lw)
    ax[0].plot(U_NRG_vector/(1./2.),  chi_sp_NRG * np.pi*Deltas_NRG, '.-', label='NRG', color='k', linewidth=0.5*lw)
    
    ax[1].plot(1/Deltas_fRG, (chi_ch_K1*np.pi*Deltas_fRG), '.-', label=labels[0], color='tab:blue', linewidth=lw)
    ax[1].plot(1/Deltas_fRG, (chi_ch_K2_1L*np.pi*Deltas_fRG), '.-', label=labels[2], color='tab:green', linewidth=lw)
    ax[1].plot(1/Deltas_fRG, (chi_ch_K2_2L*np.pi*Deltas_fRG), '.-', label=labels[3], color='tab:red', linewidth=lw)
    ax[1].plot(U_NRG_vector/(1./2.), chi_ch_NRG * np.pi*Deltas_NRG, '.-', label='NRG', color='k', linewidth=0.5*lw)
    ax[1].set_ylim(0.1, 0.6)
    
    ax[0].set_ylabel(r'${\chi^R_{\mathrm{sp}}(\omega=0)}\pi\Delta$', fontsize=fs)
    ax[1].set_ylabel(r'${\chi^R_{\mathrm{ch}}(\omega=0)}\pi\Delta$', fontsize=fs)
    
    
       
    ax[2].plot(1/Deltas_fRG, Z_K1, '.-', label=labels[0], color='tab:blue', linewidth=lw)
    ax[2].plot(1/Deltas_fRG, Z_K2_1L, '.-', label=labels[2], color='tab:green', linewidth=lw)
    ax[2].plot(1/Deltas_fRG, Z_K2_2L, '.-', label=labels[3], color='tab:red', linewidth=lw)
    ax[2].plot(2*U_NRG_vector, Z_NRG, '.-',color='k', label='NRG', linewidth=0.5*lw)
        
    ax[2].set_ylabel(r'$Z$', fontsize=fs)
    ax[2].set_xscale('log')
    
    for i in range(3):
        ax[i].xaxis.set_major_locator(MaxNLocator(5))
        ax[i].set_xlim(1/Deltas_NRG[1], 1/Deltas_NRG[-2])
        ax[i].set_xlabel(r'$U/\Delta$', fontsize=fs)
        ax[i].tick_params(axis='both', labelsize=fs-4)

    ax[2].legend(fontsize=fs-4, frameon=False)
    
    plt.tight_layout()
    plt.savefig("Error_analysis.pdf", bbox_inches='tight')
    
    
    
if diff:
    nrg_compare_chi = np.zeros(len(K1aR_0_K1))
    nrg_compare_Z = np.zeros(len(Z_K1))
    
    for index in range(len(Deltas_fRG)):
        nrg_compare_chi[index] = interpolate(chi_sp_NRG, 2*U_NRG_vector, 1/Deltas_fRG[index] )
        nrg_compare_Z[index] = interpolate(Z_NRG, 2*U_NRG_vector, 1/Deltas_fRG[index] )

        
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

    ax[0].plot(1/Deltas_fRG, (chi_sp_K1 - nrg_compare_chi)*np.pi*Deltas_fRG, '.-', label=labels[0], color='tab:blue', linewidth=lw)
    ax[0].plot(1/Deltas_fRG, (chi_sp_K2_1L - nrg_compare_chi)*np.pi*Deltas_fRG, '.-', label=labels[2], color='tab:green', linewidth=lw)
    ax[0].plot(1/Deltas_fRG, (chi_sp_K2_2L - nrg_compare_chi)*np.pi*Deltas_fRG, '.-', label=labels[3], color='tab:red', linewidth=lw)
    ax[0].set_ylabel(r'${\chi^R_{\mathrm{sp}}(\omega=0)}_{\mathrm{fRG-NRG}}\pi\Delta$', fontsize=fs)    
        
    ax[1].plot(1/Deltas_fRG, Z_K1-nrg_compare_Z, '.-', label=labels[0], color='tab:blue', linewidth=lw)
    #x[i].plot(1/Deltas_fRG, Z_K1_sf-nrg_compare, '.-', label=labels[1], color='tab:orange', linewidth=lw)
    ax[1].plot(1/Deltas_fRG, Z_K2_1L-nrg_compare_Z, '.-', label=labels[2], color='tab:green', linewidth=lw)
    ax[1].plot(1/Deltas_fRG, Z_K2_2L-nrg_compare_Z, '.-', label=labels[3], color='tab:red', linewidth=lw)
    ax[1].set_ylabel(r'$Z_{\mathrm{fRG-NRG}}$', fontsize=fs)

    
    for i in range(2):
        ax[i].set_xlim(1/Deltas_NRG[1], 1/Deltas_NRG[-2])
        ax[i].set_xlabel(r'$U/\Delta$', fontsize=fs)
        ax[i].tick_params(axis='both', labelsize=fs-4)
    
    
    ax[2].legend(fontsize=fs-4, frameon=False)
    plt.tight_layout()
    fig.savefig("Error_analysis_diff.pdf", bbox_inches='tight')
plt.show()

In [ ]:
fs = 26
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
plt.rcParams['legend.title_fontsize'] = fs
plt.rcParams["legend.columnspacing"] = '0.1'

ax[0].plot(1/Deltas_fRG, (chi_sp_K1 + chi_ch_K1)*np.pi*Deltas_fRG, '.-', label=r'$\,$', color='tab:blue', linewidth=lw)  
ax[0].plot(1/Deltas_fRG, (chi_sp_K2_1L + chi_ch_K2_1L)*np.pi*Deltas_fRG, '.-', label= r'$\,$', color='tab:green', linewidth=lw)
ax[0].plot(1/Deltas_fRG, (chi_sp_K2_2L + chi_ch_K2_2L)*np.pi*Deltas_fRG, '.-', label= r'$\,$', color='tab:red', linewidth=lw)
ax[0].plot(2*U_NRG_vector, (chi_sp_NRG + chi_ch_NRG)  * np.pi*Deltas_NRG, '.-', label=r'$\,$', color='k', linewidth=0.5*lw)

ax[0].plot(1/Deltas_fRG, 1/Z_K1, '--', label=labels[0], color='tab:blue', linewidth=lw)
ax[0].plot(1/Deltas_fRG, 1/Z_K2_1L, '--', label=labels[2], color='tab:green', linewidth=lw)
ax[0].plot(1/Deltas_fRG, 1/Z_K2_2L, '--', label=labels[3], color='tab:red', linewidth=lw)
ax[0].plot(2*U_NRG_vector, 1/Z_NRG, '--', label='NRG', color='k', linewidth=0.5*lw)

ax[1].plot(1/Deltas_fRG, (chi_sp_K1 - chi_ch_K1)*np.pi*Deltas_fRG, '.-', label=labels[0], color='tab:blue', linewidth=lw)  
ax[1].plot(1/Deltas_fRG, (chi_sp_K2_1L - chi_ch_K2_1L)*np.pi*Deltas_fRG, '.-', label=labels[2], color='tab:green', linewidth=lw)
ax[1].plot(1/Deltas_fRG, (chi_sp_K2_2L - chi_ch_K2_2L)*np.pi*Deltas_fRG, '.-', label=labels[3], color='tab:red', linewidth=lw)
ax[1].plot(2*U_NRG_vector, (chi_sp_NRG - chi_ch_NRG)  * np.pi*Deltas_NRG, '.-', label='NRG', color='k', linewidth=0.5*lw)


#ax.set_xscale('log')        

for i in range(2):
    ax[i].set_xlim(1/Deltas_NRG[1], 1/Deltas_NRG[-2])
    ax[i].set_xlabel(r'$U/\Delta$', fontsize=fs)
    ax[i].tick_params(axis='both', labelsize=fs-4)

ax[0].set_ylim(0.5, 4)
ax[1].set_ylim(0, 4)

    
ax[0].set_ylabel(r'${\chi^R_{\mathrm{e}}(\omega=0)}\pi\Delta$ , $Z^{-1}$', fontsize=fs)
ax[1].set_ylabel(r'${\chi^R_{\mathrm{o}}(\omega=0)}\pi\Delta$', fontsize=fs)
l= ax[0].legend(fontsize=fs-4, ncol=2, title=r'  $\chi_{\mathrm{e/o}}\quad \quad Z^{-1}$', frameon=False)
l._legend_box.align = "left"

plt.tight_layout()
fig.savefig("Error_analysis_ward.pdf", bbox_inches='tight')